# Distributed modeling with static schedule
We demonstrate using two workers to model 4 shots across Marmousi with a static distribution.

In [1]:
using Distributed, PyPlot

In [2]:
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [ ]:
@everywhere using DistributedArrays, DistributedJets, Jets, WaveFD, JetPackWaveFD, Random

In [4]:
v = read!("../02_marmousi_model_setup/marmousi_vp_20m_176x851.bin", Array{Float32}(undef, 176,851));
dz,dx = 20.0,20.0
nz,nx = size(v)
@show dz,dx
@show nz,nx;

(dz, dx) = (20.0, 20.0)
(nz, nx) = (176, 851)


In [5]:
sx = range(0,length=4,stop=(851-1)*20)
nshots = length(sx)
@show nshots;

nshots = 4


## Note on scratch space for temporary files
When dealing with multiple shots as in this example, we need to specify the location where scratch files will be written.

You will need to change this to point to a temporary directory available on your system.

In [1]:
scratch = "/mnt/scratch"
@assert isdir(scratch)

In [6]:
@everywhere function makeF(i,sx)
    nz,nx,dz,dx = 176,851,20.0,20.0    
    JopNlProp2DAcoIsoDenQ_DEO2_FDTD(;
        b = ones(Float32,nz,nx),
        nthreads = div(Sys.CPU_THREADS,2),
        ntrec = 1601,
        dtrec = 0.004,
        dtmod = 0.002,
        dz = dz,
        dx = dx,
        wavelet = WaveletCausalRicker(f=5.0),
        sx = sx[i],
        sz = dz,
        rx = dx*[0:1:nx-1;],
        rz = 2*dz*ones(length(0:1:nx-1)),
        reportinterval=1000)
end

In [ ]:
F = @blockop DArray(I->[makeF(i,sx) for i in I[1], j in I[2]], (nshots,1))

In [ ]:
d = F*v;

## Look at the shots

In [ ]:
shot = []
for kshot in 1:nshots
    push!(shot, getblock(d,kshot)
end
# imshow(trcs,aspect="auto",cmap="gray");clim(-1,1);colorbar();

## Remove source field files on local disk

In [ ]:
close(F)

## Remove workers

In [ ]:
rmprocs(workers())